<a href="https://colab.research.google.com/github/beruscoder/gen-AI/blob/main/agent_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [2]:
!pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [5]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=4d1d8ae207a8c845a52df8a67bf3623e&query={city}'

  response = requests.get(url)

  return response.json()

In [6]:
llm = ChatOpenAI()

In [7]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [8]:
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [10]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [11]:
response = agent_executor.invoke({"input": "Find the capital of kerala, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
We need to first find the capital of Kerala and then obtain its current weather condition.
Action: duckduckgo_search
Action Input: "capital of Kerala"Thiruvananthapuram, city, capital of Kerala state, southwestern India. It is situated along the Arabian Sea with isolated hills on a coastal plain. Padmanabhaswamy Temple One of the richest temples in the world, holding valuables estimated to be worth about $22 billion, is the Padmanabhaswamy Temple in Thiruvananthapuram (Trivandrum), Kerala ... Learn 45 fascinating facts about Thiruvananthapuram, the capital city of Kerala in India, also known as Trivandrum. Discover its rich history, cultural heritage, natural beauty, and modern development. Apart from being the capital city of Kerala, Thiruvananthapuram is a vivid combination of new and old traditions and culture. The city is situated in the southwest coast of India and has stunning views, rich history, and an array of historical sites. Below are

In [12]:
response['output']

'The capital of Kerala is Thiruvananthapuram and its current weather condition is partly cloudy with a temperature of 30°C.'